In [15]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

In [16]:
train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [17]:
train_generator = train_data_gen.flow_from_directory(
    "/mnt/d/repositories/emotion_detection/data/raw/train",
    target_size=(48,48),
    batch_size= 64,
    color_mode="grayscale",
    class_mode="categorical"
)

Found 28709 images belonging to 7 classes.


In [18]:
validation_generator = validation_data_gen.flow_from_directory(
    "/mnt/d/repositories/emotion_detection/data/raw/test",
    target_size=(48,48),
    batch_size= 64,
    color_mode="grayscale",
    class_mode="categorical"
)

Found 7178 images belonging to 7 classes.


In [19]:
emotion_model = Sequential()

# First Convolutional Block
emotion_model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(48, 48, 1)))
emotion_model.add(BatchNormalization()) 
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

# Second Convolutional Block
emotion_model.add(Conv2D(64, kernel_size=(3, 3)))
emotion_model.add(BatchNormalization()) 
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

# Third Convolutional Block
emotion_model.add(Conv2D(128, kernel_size=(3, 3)))
emotion_model.add(BatchNormalization())
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

# Fully Connected Layer
emotion_model.add(Flatten())
emotion_model.add(Dense(256, activation='relu')) 
emotion_model.add(BatchNormalization()) 
emotion_model.add(Dropout(0.5))

# Output Layer
emotion_model.add(Dense(7, activation='softmax')) 


2024-10-12 11:35:59.251096: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-12 11:35:59.407206: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-12 11:35:59.407266: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-12 11:35:59.411674: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-12 11:35:59.411736: I tensorflow/compile

In [23]:
emotion_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 46, 46, 32)        128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 23, 23, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 21, 21, 64)        2

In [26]:
emotion_model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [27]:
from tensorflow.keras.callbacks import EarlyStopping, Callback
import json

class BestModelCheckpointEveryNEpochs(Callback):
    def __init__(self, save_path_template, save_structure_path_template):
        super().__init__()
        self.save_path_template = save_path_template
        self.save_structure_path_template = save_structure_path_template
        self.best_val_loss = float('inf')  # Initialize to a large number

    def on_epoch_end(self, epoch, logs=None):
        current_val_loss = logs.get('val_loss')  # Get the current validation loss
        if current_val_loss is not None:
            # Check if the current model is the best one so far
            if current_val_loss < self.best_val_loss:
                self.best_val_loss = current_val_loss
                
                # Save the model
                filename = self.save_path_template.format(epoch=epoch + 1)
                self.model.save(filename)
                print(f'Best model saved at epoch {epoch + 1} to {filename}')
                
                # Save the model architecture as JSON
                model_json = self.model.to_json()  # Get model architecture as JSON
                structure_filename = self.save_structure_path_template.format(epoch=epoch + 1)
                with open(structure_filename, 'w') as json_file:
                    json_file.write(model_json)
                print(f'Model structure saved at epoch {epoch + 1} to {structure_filename}')

# Define the early stopping and model checkpoint callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# File path templates for saving the best model and its structure in the specified directory
save_path_template = '/mnt/d/repositories/emotion_detection/models/best_emotion_model_epoch_{epoch}.h5'  # Template for saving model
save_structure_path_template = '/mnt/d/repositories/emotion_detection/models/best_emotion_model_structure_epoch_{epoch}.json'  # Template for saving model structure

# Instantiate your custom model checkpoint
model_checkpoint = BestModelCheckpointEveryNEpochs(save_path_template, save_structure_path_template)

# Fit the model with callbacks
emotion_model_info = emotion_model.fit(
    train_generator,
    steps_per_epoch=28709 // 64,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=7178 // 64,
    callbacks=[early_stop, model_checkpoint]  # Use the custom checkpoint and early stopping
)


Epoch 1/50


2024-10-12 11:42:10.944641: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-10-12 11:42:11.382889: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2024-10-12 11:42:12.180656: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-10-12 11:42:12.440847: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f3058117390 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-12 11:42:12.440919: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-10-12 11:42:12.517464: I tensorflow/compiler/mlir/tensorflow/utils/du

448/448 [==============================] - ETA: 0s - loss: 2.1321 - accuracy: 0.2182Best model saved at epoch 1 to /mnt/d/repositories/emotion_detection/models/best_emotion_model_epoch_1.h5
Model structure saved at epoch 1 to /mnt/d/repositories/emotion_detection/models/best_emotion_model_structure_epoch_1.json
448/448 [==============================] - 107s 229ms/step - loss: 2.1321 - accuracy: 0.2182 - val_loss: 1.8245 - val_accuracy: 0.2493
Epoch 2/50


/home/saivardhan4694/miniconda3/envs/myenv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


448/448 [==============================] - 99s 221ms/step - loss: 1.8193 - accuracy: 0.2640 - val_loss: 1.8555 - val_accuracy: 0.2973
Epoch 3/50
448/448 [==============================] - 94s 211ms/step - loss: 1.7578 - accuracy: 0.2863 - val_loss: 1.9553 - val_accuracy: 0.3117
Epoch 4/50
448/448 [==============================] - ETA: 0s - loss: 1.7318 - accuracy: 0.2996Best model saved at epoch 4 to /mnt/d/repositories/emotion_detection/models/best_emotion_model_epoch_4.h5
Model structure saved at epoch 4 to /mnt/d/repositories/emotion_detection/models/best_emotion_model_structure_epoch_4.json
448/448 [==============================] - 97s 216ms/step - loss: 1.7318 - accuracy: 0.2996 - val_loss: 1.6171 - val_accuracy: 0.3905
Epoch 5/50
448/448 [==============================] - 99s 222ms/step - loss: 1.7002 - accuracy: 0.3216 - val_loss: 1.6793 - val_accuracy: 0.3491
Epoch 6/50
448/448 [==============================] - ETA: 0s - loss: 1.6770 - accuracy: 0.3300Best model saved at epo

Althogh the early stopping as triggerd a stop in training, based on my observations on the loss and accuracy values. i have decided to continue tarining since the loss is although fluctuating but still decreasing and accuracy is increasing.

In [28]:
from keras.models import load_model
from keras.callbacks import EarlyStopping

# Load the last saved model
model = load_model('/mnt/d/repositories/emotion_detection/models/best_emotion_model_epoch_24.h5')

# Set up the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

emotion_model_info = model.fit(
    train_generator,
    steps_per_epoch=28709 // 64,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=7178 // 64,
    callbacks=[early_stop, model_checkpoint]  # Use the custom checkpoint and early stopping
)

Epoch 1/20


2024-10-12 13:09:38.717451: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


448/448 [==============================] - 133s 289ms/step - loss: 1.4968 - accuracy: 0.4201 - val_loss: 1.3102 - val_accuracy: 0.5184
Epoch 2/20
448/448 [==============================] - 1403s 3s/step - loss: 1.4884 - accuracy: 0.4265 - val_loss: 1.2939 - val_accuracy: 0.5092
Epoch 3/20
448/448 [==============================] - 118s 264ms/step - loss: 1.4769 - accuracy: 0.4303 - val_loss: 1.3274 - val_accuracy: 0.4948
Epoch 4/20
448/448 [==============================] - ETA: 0s - loss: 1.5013 - accuracy: 0.4217Best model saved at epoch 4 to /mnt/d/repositories/emotion_detection/models/best_emotion_model_epoch_4.h5
Model structure saved at epoch 4 to /mnt/d/repositories/emotion_detection/models/best_emotion_model_structure_epoch_4.json
448/448 [==============================] - 118s 262ms/step - loss: 1.5013 - accuracy: 0.4217 - val_loss: 1.2727 - val_accuracy: 0.5166
Epoch 5/20
448/448 [==============================] - 117s 262ms/step - loss: 1.4722 - accuracy: 0.4321 - val_loss: 